<a href="https://colab.research.google.com/github/Ahny678/Performance-Analysis-of-Lightweight-CNN-models-/blob/main/PERFORMANCE_ANALYSIS_ON_CROP_DS_CODE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

IMPORTING AND PROCESSING DATA...

In [ ]:
import os
import numpy as np
import tensorflow as tf
from google.colab import drive
from sklearn.utils.class_weight import compute_class_weight

In [ ]:
# Mount Google Drive
drive.mount('/content/drive')

In [ ]:
# Define dataset paths
base_dir = '/content/drive/MyDrive/PERFORMANCECROPDATASET'
maize_dir = os.path.join(base_dir, 'Maize')
rice_dir = os.path.join(base_dir, 'Rice')
sorghum_dir = os.path.join(base_dir, 'Sorghum')

# Image parameters
IMG_SIZE = (224, 224)
BATCH_SIZE = 16

In [ ]:
# Function to verify dataset structure
def verify_dataset_structure(dataset_dir, dataset_name):
    print(f"\nVerifying {dataset_name} dataset structure...")
    for split in ['train', 'validation', 'test']:
        split_dir = os.path.join(dataset_dir, split)
        if not os.path.exists(split_dir):
            print(f"Error: {split_dir} does not exist!")
            continue
        classes = os.listdir(split_dir)
        print(f"{split.capitalize()} set: {len(classes)} classes")
        for cls in classes:
            cls_dir = os.path.join(split_dir, cls)
            num_images = len(os.listdir(cls_dir))
            print(f"  {cls}: {num_images} images")

In [ ]:
# Verify dataset structure for all crops

# verify_dataset_structure(maize_dir, "Maize")
# verify_dataset_structure(rice_dir, "Rice")
# verify_dataset_structure(sorghum_dir, "Sorghum")

In [ ]:
# Data augmentation for training
data_augmentation = tf.keras.Sequential([
    tf.keras.layers.RandomFlip("horizontal"),
    tf.keras.layers.RandomRotation(0.2),
    tf.keras.layers.RandomZoom(0.2),
    tf.keras.layers.RandomTranslation(0.1, 0.1),
])


In [ ]:
# Function to load dataset
def load_dataset(dataset_dir, split, shuffle=False):
    dataset = tf.keras.preprocessing.image_dataset_from_directory(
        os.path.join(dataset_dir, split),
        image_size=IMG_SIZE,
        batch_size=BATCH_SIZE,
        shuffle=shuffle,
      label_mode='categorical'  # One-hot encoded labels
    )
    # Normalize to [0, 1]
    dataset = dataset.map(lambda x, y: (x / 255.0, y), num_parallel_calls=tf.data.AUTOTUNE)
    if split == 'train':
        dataset = dataset.map(lambda x, y: (data_augmentation(x, training=True), y),
                             num_parallel_calls=tf.data.AUTOTUNE)
    # Convert to float32
    dataset = dataset.map(lambda x, y: (tf.cast(x, tf.float32), y),
                         num_parallel_calls=tf.data.AUTOTUNE)
    # Cache and prefetch for performance
    dataset = dataset.cache().prefetch(tf.data.AUTOTUNE)
    return dataset

In [ ]:
# Load datasets for each crop
maize_train = load_dataset(maize_dir, 'train', shuffle=True)
maize_val = load_dataset(maize_dir, 'validation')
maize_test = load_dataset(maize_dir, 'test')

rice_train = load_dataset(rice_dir, 'train', shuffle=True)
rice_val = load_dataset(rice_dir, 'validation')
rice_test = load_dataset(rice_dir, 'test')

sorghum_train = load_dataset(sorghum_dir, 'train', shuffle=True)
sorghum_val = load_dataset(sorghum_dir, 'validation')
sorghum_test = load_dataset(sorghum_dir, 'test')

Found 2950 files belonging to 4 classes.
Found 836 files belonging to 4 classes.
Found 422 files belonging to 4 classes.
Found 1830 files belonging to 6 classes.
Found 389 files belonging to 6 classes.
Found 400 files belonging to 6 classes.
Found 5439 files belonging to 7 classes.
Found 1560 files belonging to 7 classes.
Found 782 files belonging to 7 classes.


In [ ]:
# Function to compute class weights
def compute_class_weights(dataset_dir, split, class_names):
    class_counts = {}
    split_dir = os.path.join(dataset_dir, split)
    for cls in class_names:
        cls_dir = os.path.join(split_dir, cls)
        class_counts[cls] = len(os.listdir(cls_dir))

    # Get class indices
    labels = []
    for cls in class_names:
        count = class_counts[cls]
        labels.extend([class_names.index(cls)] * count)

    # Compute class weights
    weights = compute_class_weight(
        class_weight='balanced',
        classes=np.unique(labels),
        y=labels
    )
    return dict(zip(range(len(class_names)), weights))

In [ ]:
# Get class names for each dataset
maize_class_names = sorted(os.listdir(os.path.join(maize_dir, 'train')))
rice_class_names = sorted(os.listdir(os.path.join(rice_dir, 'train')))
sorghum_class_names = sorted(os.listdir(os.path.join(sorghum_dir, 'train')))

# Compute class weights for training sets
maize_class_weights = compute_class_weights(maize_dir, 'train', maize_class_names)
rice_class_weights = compute_class_weights(rice_dir, 'train', rice_class_names)
sorghum_class_weights = compute_class_weights(sorghum_dir, 'train', sorghum_class_names)

In [ ]:
# Print class weights
print("\nMaize Class Weights:", maize_class_weights)
print("Rice Class Weights:", rice_class_weights)
print("Sorghum Class Weights:", sorghum_class_weights)



Maize Class Weights: {0: np.float64(0.919576059850374), 1: np.float64(0.7981601731601732), 2: np.float64(1.839152119700748), 3: np.float64(0.8961117861482382)}
Rice Class Weights: {0: np.float64(1.0166666666666666), 1: np.float64(0.9967320261437909), 2: np.float64(0.9967320261437909), 3: np.float64(0.9967320261437909), 4: np.float64(0.9967320261437909), 5: np.float64(0.9967320261437909)}
Sorghum Class Weights: {0: np.float64(1.0959097320169253), 1: np.float64(0.9098360655737705), 2: np.float64(3.9846153846153847), 3: np.float64(2.226361031518625), 4: np.float64(0.4666666666666667), 5: np.float64(1.85), 6: np.float64(0.6230954290296712)}


In [ ]:
import matplotlib.pyplot as plt

# Function to plot class weights
def plot_class_weights(class_weights, class_names, crop_name):
    plt.figure(figsize=(10, 6))
    weights = list(class_weights.values())
    indices = list(class_weights.keys())

    plt.bar(class_names, weights, color='skyblue')
    plt.xlabel('Class Name')
    plt.ylabel('Class Weight')
    plt.title(f'{crop_name} Class Weights')
    plt.xticks(rotation=45)
    plt.grid(axis='y', linestyle='--', alpha=0.7)
    plt.tight_layout()
    plt.show()


In [ ]:
# # Plot for Maize
plot_class_weights(maize_class_weights, maize_class_names, "Maize")

In [ ]:
# # Plot for Rice
plot_class_weights(rice_class_weights, rice_class_names, "Rice")



In [ ]:
# # Plot for Sorghum
plot_class_weights(sorghum_class_weights, sorghum_class_names, "Sorghum")

In [ ]:
# eg: Check a batch from maize_train
for images, labels in maize_train.take(1):
    print("\nSample batch from Maize Train:")
    print("Image shape:", images.shape)
    print("Label shape:", labels.shape)
    print("Pixel range:", tf.reduce_min(images).numpy(), tf.reduce_max(images).numpy())

In [ ]:
# Import additional libraries
from tensorflow.keras.applications import VGG16, MobileNetV3Small, DenseNet121, NASNetMobile, EfficientNetB2
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt

In [ ]:
# Define output directory for saving models and plots
output_dir = '/content/drive/MyDrive/crop_model_results'
os.makedirs(output_dir, exist_ok=True)

# Training parameters
EPOCHS = 3

In [ ]:
# Model configurations
models_config = {
    'VGG16': {
        'base_model': VGG16,
        'preprocess_fn': tf.keras.applications.vgg16.preprocess_input
    },
    'MobileNetV3': {
        'base_model': MobileNetV3Small,
        'preprocess_fn': tf.keras.applications.mobilenet_v3.preprocess_input
    },
    'DenseNet121': {
        'base_model': DenseNet121,
        'preprocess_fn': tf.keras.applications.densenet.preprocess_input
    },
    'NasNetMobile': {
        'base_model': NASNetMobile,
        'preprocess_fn': tf.keras.applications.nasnet.preprocess_input
    },
    'EfficientNetB2': {
        'base_model': EfficientNetB2,
        'preprocess_fn': tf.keras.applications.efficientnet.preprocess_input
    }
}

In [ ]:
# # Function to build model
def build_model(base_model_fn, preprocess_fn, num_classes, model_name):
    base_model = base_model_fn(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
    base_model.trainable = False  # Freeze base model
    inputs = tf.keras.Input(shape=(224, 224, 3))
    x = preprocess_fn(inputs)  # Apply model-specific preprocessing
    x = base_model(x, training=False)
    x = GlobalAveragePooling2D()(x)
    outputs = Dense(num_classes, activation='softmax')(x)
    model = Model(inputs, outputs, name=model_name)
    model.compile(
        optimizer=Adam(learning_rate=0.001),
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )
    return model

In [ ]:
import json
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, confusion_matrix

def train_and_evaluate(train_ds, val_ds, test_ds, class_weights, num_classes, model_name, crop_name, preprocess_fn, class_names):
    # Setup organized directories
    crop_dir = os.path.join(output_dir, crop_name)
    model_dir = os.path.join(crop_dir, model_name)
    os.makedirs(model_dir, exist_ok=True)

    # Build model
    base_model_fn = models_config[model_name]['base_model']
    model = build_model(base_model_fn, preprocess_fn, num_classes, f"{crop_name}_{model_name}")

    # Preprocess datasets
    train_ds_proc = train_ds.map(lambda x, y: (preprocess_fn(x * 255.0), y), num_parallel_calls=tf.data.AUTOTUNE)
    val_ds_proc = val_ds.map(lambda x, y: (preprocess_fn(x * 255.0), y), num_parallel_calls=tf.data.AUTOTUNE)
    test_ds_proc = test_ds.map(lambda x, y: (preprocess_fn(x * 255.0), y), num_parallel_calls=tf.data.AUTOTUNE)

    train_ds_proc = train_ds_proc.cache().shuffle(1000).prefetch(tf.data.AUTOTUNE)
    val_ds_proc = val_ds_proc.cache().prefetch(tf.data.AUTOTUNE)
    test_ds_proc = test_ds_proc.cache().prefetch(tf.data.AUTOTUNE)

    # Checkpoint to save best model weights
    checkpoint_path = os.path.join(model_dir, "best_model.h5")
    checkpoint = tf.keras.callbacks.ModelCheckpoint(
        checkpoint_path, monitor='val_accuracy', save_best_only=True, mode='max', verbose=1
    )

    # Train
    history = model.fit(
        train_ds_proc,
        validation_data=val_ds_proc,
        epochs=EPOCHS,
        class_weight=class_weights,
        callbacks=[checkpoint],
        verbose=1
    )

    # Load best weights
    model.load_weights(checkpoint_path)

    # Save complete model
    model.save(os.path.join(model_dir, "full_model.keras"))

    # Evaluate on test data
    test_loss, test_accuracy = model.evaluate(test_ds_proc)
    print(f"\n{crop_name} {model_name} Test Accuracy: {test_accuracy:.4f}")

    # Predictions
    y_true, y_pred = [], []
    for images, labels in test_ds_proc:
        preds = model.predict(images)
        y_true.extend(np.argmax(labels.numpy(), axis=1))
        y_pred.extend(np.argmax(preds, axis=1))

    # Classification report
    report_dict = classification_report(y_true, y_pred, target_names=class_names, output_dict=True)
    with open(os.path.join(model_dir, "classification_report.json"), "w") as f:
        json.dump(report_dict, f, indent=4)

    # Confusion matrix
    cm = confusion_matrix(y_true, y_pred)
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', xticklabels=class_names, yticklabels=class_names, cmap="Blues")
    plt.title(f"{crop_name} {model_name} Confusion Matrix")
    plt.xlabel("Predicted")
    plt.ylabel("True")
    plt.tight_layout()
    plt.savefig(os.path.join(model_dir, "confusion_matrix.png"))
    plt.close()

    # Training history plots
    plt.figure(figsize=(12, 4))
    plt.subplot(1, 2, 1)
    plt.plot(history.history['accuracy'], label='Train Accuracy')
    plt.plot(history.history['val_accuracy'], label='Val Accuracy')
    plt.title('Accuracy over Epochs')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.legend()

    plt.subplot(1, 2, 2)
    plt.plot(history.history['loss'], label='Train Loss')
    plt.plot(history.history['val_loss'], label='Val Loss')
    plt.title('Loss over Epochs')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()

    plt.tight_layout()
    plt.savefig(os.path.join(model_dir, "training_plot.png"))
    plt.close()

    # Save training history
    with open(os.path.join(model_dir, "training_history.json"), "w") as f:
        json.dump(history.history, f, indent=4)

    return model, history


In [ ]:
# Define crops and their datasets
crops = [
    {
        'name': 'Maize',
        'train_ds': maize_train,
        'val_ds': maize_val,
        'test_ds': maize_test,
        'class_weights': maize_class_weights,
        'num_classes': len(maize_class_names),
        'class_names': maize_class_names
    },
    {
        'name': 'Rice',
        'train_ds': rice_train,
        'val_ds': rice_val,
        'test_ds': rice_test,
        'class_weights': rice_class_weights,
        'num_classes': len(rice_class_names),
        'class_names': rice_class_names
    },
    {
        'name': 'Sorghum',
        'train_ds': sorghum_train,
        'val_ds': sorghum_val,
        'test_ds': sorghum_test,
        'class_weights': sorghum_class_weights,
        'num_classes': len(sorghum_class_names),
        'class_names': sorghum_class_names
    }
]


MAIZE-VGG16

In [ ]:
model_name = 'VGG16'
crop = crops[0]  # Maize

model, history = train_and_evaluate(
    crop['train_ds'], crop['val_ds'], crop['test_ds'],
    crop['class_weights'], crop['num_classes'],
    model_name, crop['name'],
    models_config[model_name]['preprocess_fn'],
    crop['class_names']
)


MAIZE-MOBILENETV3

In [ ]:
# model_name = 'MobileNetV3'
# crop = crops[0]  # Maize

# model, history = train_and_evaluate(
#     crop['train_ds'], crop['val_ds'], crop['test_ds'],
#     crop['class_weights'], crop['num_classes'],
#     model_name, crop['name'],
#     models_config[model_name]['preprocess_fn'],
#     crop['class_names']
# )


Maize - DenseNet121

In [ ]:
# model_name = 'DenseNet121'
# crop = crops[0]  # Maize

# model, history = train_and_evaluate(
#     crop['train_ds'], crop['val_ds'], crop['test_ds'],
#     crop['class_weights'], crop['num_classes'],
#     model_name, crop['name'],
#     models_config[model_name]['preprocess_fn'],
#     crop['class_names']
# )


Maize - NasNetMobile

In [ ]:
# model_name = 'NasNetMobile'
# crop = crops[0]  # Maize

# model, history = train_and_evaluate(
#     crop['train_ds'], crop['val_ds'], crop['test_ds'],
#     crop['class_weights'], crop['num_classes'],
#     model_name, crop['name'],
#     models_config[model_name]['preprocess_fn'],
#     crop['class_names']
# )


Maize - EfficientNetB2

In [ ]:
# model_name = 'EfficientNetB2'
# crop = crops[0]  # Maize

# model, history = train_and_evaluate(
#     crop['train_ds'], crop['val_ds'], crop['test_ds'],
#     crop['class_weights'], crop['num_classes'],
#     model_name, crop['name'],
#     models_config[model_name]['preprocess_fn'],
#     crop['class_names']
# )


Rice - VGG16

In [ ]:
# model_name = 'VGG16'
# crop = crops[1]  # Rice

# model, history = train_and_evaluate(
#     crop['train_ds'], crop['val_ds'], crop['test_ds'],
#     crop['class_weights'], crop['num_classes'],
#     model_name, crop['name'],
#     models_config[model_name]['preprocess_fn'],
#     crop['class_names']
# )


Rice - MobileNetV3

In [ ]:
# model_name = 'MobileNetV3'
# crop = crops[1]  # Rice

# model, history = train_and_evaluate(
#     crop['train_ds'], crop['val_ds'], crop['test_ds'],
#     crop['class_weights'], crop['num_classes'],
#     model_name, crop['name'],
#     models_config[model_name]['preprocess_fn'],
#     crop['class_names']
# )


Rice - DenseNet121

In [ ]:
# model_name = 'DenseNet121'
# crop = crops[1]  # Rice

# model, history = train_and_evaluate(
#     crop['train_ds'], crop['val_ds'], crop['test_ds'],
#     crop['class_weights'], crop['num_classes'],
#     model_name, crop['name'],
#     models_config[model_name]['preprocess_fn'],
#     crop['class_names']
# )


Rice - NasNetMobile

In [ ]:
# model_name = 'NasNetMobile'
# crop = crops[1]  # Rice

# model, history = train_and_evaluate(
#     crop['train_ds'], crop['val_ds'], crop['test_ds'],
#     crop['class_weights'], crop['num_classes'],
#     model_name, crop['name'],
#     models_config[model_name]['preprocess_fn'],
#     crop['class_names']
# )


Rice - EfficientNetB2

In [ ]:
# model_name = 'EfficientNetB2'
# crop = crops[1]  # Rice

# model, history = train_and_evaluate(
#     crop['train_ds'], crop['val_ds'], crop['test_ds'],
#     crop['class_weights'], crop['num_classes'],
#     model_name, crop['name'],
#     models_config[model_name]['preprocess_fn'],
#     crop['class_names']
# )

 Sorghum - VGG16

In [ ]:
# model_name = 'VGG16'
# crop = crops[2]  # Sorghum

# model, history = train_and_evaluate(
#     crop['train_ds'], crop['val_ds'], crop['test_ds'],
#     crop['class_weights'], crop['num_classes'],
#     model_name, crop['name'],
#     models_config[model_name]['preprocess_fn'],
#     crop['class_names']
# )
# DIDNT COMPLETE

 Sorghum - MobileNetV3

In [ ]:
# model_name = 'MobileNetV3'
# crop = crops[2]  # Sorghum

# model, history = train_and_evaluate(
#     crop['train_ds'], crop['val_ds'], crop['test_ds'],
#     crop['class_weights'], crop['num_classes'],
#     model_name, crop['name'],
#     models_config[model_name]['preprocess_fn'],
#     crop['class_names']
# )


Sorghum - DenseNet121

In [ ]:
# model_name = 'DenseNet121'
# crop = crops[2]  # Sorghum

# model, history = train_and_evaluate(
#     crop['train_ds'], crop['val_ds'], crop['test_ds'],
#     crop['class_weights'], crop['num_classes'],
#     model_name, crop['name'],
#     models_config[model_name]['preprocess_fn'],
#     crop['class_names']
# )


 Sorghum - NasNetMobile

In [ ]:
# model_name = 'NasNetMobile'
# crop = crops[2]  # Sorghum

# model, history = train_and_evaluate(
#     crop['train_ds'], crop['val_ds'], crop['test_ds'],
#     crop['class_weights'], crop['num_classes'],
#     model_name, crop['name'],
#     models_config[model_name]['preprocess_fn'],
#     crop['class_names']
# )


Sorghum - EfficientNetB2

In [ ]:
# model_name = 'EfficientNetB2'
# crop = crops[2]  # Sorghum

# model, history = train_and_evaluate(
#     crop['train_ds'], crop['val_ds'], crop['test_ds'],
#     crop['class_weights'], crop['num_classes'],
#     model_name, crop['name'],
#     models_config[model_name]['preprocess_fn'],
#     crop['class_names']
# )
